uv add playwright beautifulsoup4
playwright install

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
import os
from IPython.display import display, Image
import requests
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.agents import Tool
import gradio as gr
import nest_asyncio
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

nest_asyncio.apply()
load_dotenv(override=True)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
async_browser = create_async_playwright_browser(headless=False)
toolkit = PlayWrightBrowserToolkit(async_browser=async_browser)
tools = toolkit.get_tools()

# for tool in tools:
#     print(f"{tool.name}: {tool.description}")

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State) -> State:
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools_node", ToolNode(tools=tools))
graph_builder.add_conditional_edges("chatbot", tools_condition, {"tools": "tools_node", END: END})
graph_builder.add_edge("tools_node", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
async def chat(user_input: str, history):
    result = await graph.ainvoke({"messages": history + [{ "role": "user", "content": user_input }]})
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()